In [36]:
import os
import random
import shutil

In [2]:
class SplitDataSet(object):
    def __init__(self, data_dir):
        self.data_dir = data_dir
    
    '''
    这个是用于分类数据集的，即随机选择一定比例的图片，
    用于训练和验证；
    arg: data_dir: 分类数据集的路径
    '''
        
    def read_all_pic(self):
        ntxt, n2txt = ntxt.txt, n2txt.txt
        for root, dirs, _ in os.walk(self.data_dir):
            if dirs == 'reshape-2n':
                root_2n_path = os.path.join(root, dirs)
                n2_files = [os.path.join(root_2n_path, n2) for n2 in os.listdir(root_n_path)]
                with open(n2txt, 'w') as f:
                    for file in n_files:
                        f.write(file + '\n')
            if dirs == 'reshape-n':
                root_n_path = os.path.join(root, dirs)
                n_files = [os.path.join(root_n_path, n) for n in os.listdir(root_n_path)]
                with open(ntxt, 'w') as f:
                    for file in n_files:
                        f.write(file + '\n')
                
    def splitdata(self, ntxt, n2txt, ratio):
        with open(ntxt, 'r') as f:
            lines = f.readlines()
        with open(n2txt, 'r') as f:
            lines2 = f.readlines()
        n_files_train_num = int((len(lines) * ratio)//1)
        n2_files_train_num = int((len(lines2) * ratio)//1)
        n_files_train = random.sample(lines, n_files_train_num)
        n2_files_train = random.sample(lines2, n2_files_train_num)
        n_files_val = [line for line in lines if line not in n_files_train]
        n2_files_val = [line for line in lines2 if line not in n2_files_train]
        assert len(n_files_train) + len(n2_files_train) + \
            len(n_files_val) + len(n2_files_val) == len(lines) + len(lines2), \
            'The number of training and validation files do not match.'
        return n_files_train, n2_files_train, n_files_val, n2_files_val
    
    def split_train_val_2dir(self, root, ntxt, n2txt, ratio):
        n_files_train, n2_files_train, n_files_val, n2_files_val = self.splitdata(
            ntxt, n2txt, ratio)
        for i in range(len(n_files_train)):
            shutil.copy(n_files_train[i].strip(), root + 'train/n/')
        for train_2n_file in n2_files_train:
            shutil.copy(train_2n_file.strip(), root + 'train/2n/')
        for val_2n_file in n2_files_val:
            shutil.copy(val_2n_file.strip(), root + 'val/2n/')
        for val_n_file in n_files_val:
            shutil.copy(val_n_file.strip(), root + 'val/n/')
        

In [4]:
# SplitDataSet(data_dir=r'./qk_data/class_data/').split_train_val_2dir(
#                         root=r'./qk_data/class_data/',
#                          ntxt='./n.txt',n2txt='./2n.txt', ratio=.8)

In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import pandas as pd

# 定义数据集路径和预处理方法
data_dir = './qk_data/class_data'
train_dir = data_dir + '/train'
val_dir = data_dir + '/val'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

# 定义训练集、验证集和测试集
train_dataset = ImageFolder(train_dir, transform=train_transform)
val_dataset = ImageFolder(val_dir, transform=val_transform)

# 定义数据加载器
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# 训练模型
def train_model(epochs, model, train_loader, val_loader, csv_name, lr=.01, mt=.0009):
    criterion = nn.CrossEntropyLoss(reduce='mean')
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=mt)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    running_loss = []
    acc = []
    val_loss = []
    for epoch in range(epochs):

        train_loss = 0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            if i % 100 == 99:
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, train_loss / 100))
        running_loss.append(train_loss / i+1)
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            vloss = 0
            for data in val_loader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                l = criterion(outputs, labels)
                vloss += l.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))
            acc.append(100 * correct / total)
            val_loss.append(vloss / i+1)
    df_loss = pd.DataFrame(running_loss)
    df_acc = pd.DataFrame(acc)  
    df_vloss = pd.DataFrame(val_loss)
    df_acc.to_csv(csv_name + '_acc.csv', index=False)
    df_loss.to_csv(csv_name + '_loss.csv', index=False)
    df_vloss.to_csv(csv_name + '_vloss.csv', index=False)
    return running_loss, acc, val_loss


In [38]:

# 定义r50模型
model_r50 = models.resnet50(pretrained=True)
num_features = model_r50.fc.in_features
model_r50.fc = nn.Linear(num_features, 2) # 分类器,2表示n和2n

# 定义r34模型
model_r34 = models.resnet34(pretrained=True)
num_features = model_r34.fc.in_features
model_r34.fc = nn.Linear(num_features, 2)

# 定义Vgg16模型
model_vgg16 = models.vgg16(pretrained=True)
model_vgg16.classifier.add_module('qk_classier', nn.Linear(1000, 2))

# 定义mobilenet_v3_Large模型
model_mobilenet_v3_large = models.mobilenet_v3_large(pretrained=True)
model_mobilenet_v3_large.classifier.add_module('qk_classier', nn.Linear(1000, 2))

# 定义efficient_b0模型

model_efficient_b0 = models.efficientnet_b0(pretrained=True)
model_efficient_b0.classifier.add_module('qk_classier', nn.Linear(1000,2))

c:\Users\16477\anaconda3\envs\hy-dl\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\16477\anaconda3\envs\hy-dl\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\16477\anaconda3\envs\hy-dl\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weight

In [41]:
# train_model(model=model_mobilenet_v3_large, epochs=2, train_loader=train_loader, 
#              val_loader=val_loader, lr=0.001, mt=0.009, csv_name='mobilenet_v3_large');

c:\Users\16477\anaconda3\envs\hy-dl\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


Accuracy of the network on the test images: 68 %
Accuracy of the network on the test images: 79 %
